In [2]:
import os
import sys
import csv
import copy
import random
import itertools
from operator import itemgetter
from collections import defaultdict
import pprint

# Make sure you've got Numpy and Scipy installed:
import numpy as np
import scipy
import scipy.spatial.distance
from numpy.linalg import svd

import matplotlib.pyplot as plt
import gensim
import fileinput
from glob import glob

from bs4 import BeautifulSoup as soup

In [ ]:
%matplotlib inline

## Distributional Matrices

We can start by creating a dictionary $d$ that can be used to map word-pairs to counts. Everytime a pair of words $w$ and $w^\prime$ occur, we can increment a counter associated with each pair $d[w,w^\prime]$ by $1$. Using these count dictionary we can then create our vocabulary $V$, an ordered list of words types.

We can then create a matrix, $M$, of dimensions $|V|$ x $|V|$. Each $M[i,j]$ is filled with the counts contained in $d[w_i,w_j]$.

These co-occurence matrices have been provided with this example. We can import them with the _build_ function seen below.

In [ ]:
# Build method to import co-occurence matrices

def build(src_filename, delimiter=',', header=True, quoting=csv.QUOTE_MINIMAL):    
    reader = csv.reader(open(src_filename), delimiter=delimiter, quoting=quoting)
    colnames = None
    if header:
        colnames = reader.__next__()
        colnames = colnames[1: ]
    mat = []    
    rownames = []
    for line in reader:
        rownames.append(line[0])            
        mat.append(np.array(list(map(float, line[1: ]))))    
    return (np.array(mat), rownames, colnames)


We can now read in the example co-occurency matrices for later. We will be using the IMDB moview review dataset for these examples.


In [ ]:
# Import word <-> word co-occurence matrix
ww = build('distributedwordreps-data/imdb-wordword.csv')

#Import w <-> document co-occurence matrix
wd = build('distributedwordreps-data/imdb-worddoc.csv')

In [ ]:
ww[0][0][0]

In [ ]:
# Example of counts of first words in the document which happen to be two exclamation marks

print("Word co-occurences : " + ww[1][0] + ", and, " + ww[2][0])

print("Count : " + str(ww[0][0][0]))

In [ ]:
# Load GloVe vectors as well

glv = build('distributedwordreps-data/glove.6B.50d.txt', delimiter=' ', header=False, quoting=csv.QUOTE_NONE)

# Vector Comparisons

For the most part we are interested in measuring the _distance_ between two vectors. The general idea of vector comparisons using distance is that words that are semantically similar should be closer together in the vector spaces we build, and symantically unrelated words should be further apart.

The scipy library has a lot of vector comparison models methods, and for the purposes of our work here we'll be using this library as a supporting implementation of these methods.

### Euclidean Distance

The first comparison methodology we'll look at is Euclidean Distance. To find this distance, 

The equation to find the Euclidean Distance between vectors $u$ and $v$ of $n$ dimensions is below :

$$\sqrt{\sum_{i=1}^{n} |u_{i}-v_{i}|^2}$$


In two dimensions, this corresponds to the length of the direct most line between the two points.

Below is a method to define that function :

In [ ]:
# method to compute euclidean distance
# we exploit a method already defined in scipy

def euclidean(u, v):
    return scipy.spatial.distance.euclidean(u, v)

Here I create a small toy array (matrix) for use within our examples.

In [ ]:
# Two dimensional vector embedding in two dimensional space to show 
# how we can measure the distance between two vectors in 2 space

ABC = np.array([
    [ 2.0,  4.0],  # 0
    [ 1.50, 3.50], # 1
    [10.0, 15.0],  # 2
    [14.0, 10.0]]) # 3

def plot_ABC(m):
    plt.plot(m[:,0], m[:,1], marker='', linestyle='')
    plt.xlim([0,np.max(m)*1.2])
    plt.ylim([0,np.max(m)*1.2])
    for i, x in enumerate(['0','1','2','3']):
        plt.annotate(x, m[i,:])

plot_ABC(ABC)

In [ ]:
#Measure the distance between vectors A and B in matrix ABC
euclidean(ABC[0], ABC[1])

In [ ]:
#Measure the distance between vectors B and C in matrix ABC
euclidean(ABC[0], ABC[1])

## Vector Length

Euclidean distance measures the difference between two vector lengths, as shown the in the equation above. We're also able to find the length of a single vector, for use in later examples, as well by using the following equation :

$$\|u\| = \sqrt{\sum_{i=1}^{n} u_{i}^{2}}$$

We use the numpy library sqrt and dot product methods within the library shown below.

In [ ]:
def vector_length(u):
    return np.sqrt(np.dot(u, u))

## Length Normalization

When working with datasets that have large(r), variation in the size of the datapoints that are used can skew the actual distance measured between vectors. Below we define a normalization function that will normalize each vector according to its length.

When normalizing all vectors according to their length, we can see in the plot below that this changes the representation of the data quite and bit and actually brings vector 0 and 1 closer together on the plot, showing their stronger similarity than say vectors 2 and 3, or 0 and 3.

In [ ]:
def length_norm(u):
    return  u / vector_length(u)

In [ ]:
plot_ABC(np.array([length_norm(row) for row in ABC]))

### Cosine Distance

The cosine distance takes the overall vector length into account, meaning we don't have to run the vector_length() method over the vectors before calculation, and measure the angle between the two vectors. This is all captured within the Cosine Distance measurement function that is seen below :

$$1 - \left(\frac{\sum_{i=1}^{n} u_{i} \cdot v_{i}}{\|u\|\cdot \|v\|}\right)$$

The result is the same as first normalizing the vectors according to their length, vector_length(), and then computing the Euclidean distance between the two.

In [ ]:
def cosine(u, v):
    # Use scipy's method:
    return scipy.spatial.distance.cosine(u, v)

In [ ]:
cosine(ABC[1],ABC[3])

## Distributional Neighbors

This functional is an investigational aide and allows us to use the similarity functions defined above over the IMDB dataset that we've previously loaded.

This function is performing a cosine similarity computation ranking the _rownames_ according to their distance from _word_, as measured by the distfunc in matrix mat.

The function returns a sorted list of words that ascend from the closest, distance wise, depending on which distance method is passed in. The method defaults to using the cosine distance measurement.

In [ ]:
def neighbors(word=None, mat=None, rownames=None, distfunc=cosine):
    if word not in rownames:
        raise ValueError('%s is not in this dataset' % word)
    w = mat[rownames.index(word)]
    dists = [(rownames[i], distfunc(w, mat[i])) for i in range(len(mat))]
    return sorted(dists, key=itemgetter(1), reverse=False)

The neighbors method above might be a bit confusing, as its performing the cosine calculation over all vectors of co-occurences, per pair, within the dataset that's provided. And returning a sorted list of closest words to the input word.

That said, here I provide a simple single word comparison that is then extrapolated across all words in the co-occurance matrix, within the neighbors function.

In [ ]:
w1 = ww[0][ww[1].index('superb')]

w2 = ww[0][ww[1].index('wonderfully')]

print("The vector representation for w1 is : " + str(w1))

print("The vector representation for w2 is : " + str(w2))

print("The cosine distance between vectors w1 and w2 is : " + str(cosine(w1, w2)))

Breaking this down even further, instead of allowing the cosine distance function to perform the normalization, and Euclidean distance calculations, we can do it by hand.

In [ ]:
# Generic manual calculation of the Euclidean distance with the dot product of both vectors divided by
# the length of the vectors, also known as the L2 norm of those vectors
manual = 1.0 - (np.dot(w1, w2) / (vector_length(w1) * vector_length(w2)))

manual

Here we show an example of a word and their respective neighbors according to both cosine similarity and basic euclidean distance.

In [ ]:
neighbors(word='superb', mat=ww[0], rownames=ww[1], distfunc=cosine)[: 10]

In [ ]:
neighbors(word='superb', mat=ww[0], rownames=ww[1], distfunc=euclidean)[: 10]

### GloVe representation

Here we can see that the GloVe vectors perform _very_ well, but they are based on much more than a simple raw count of occurences in the text.

In [ ]:
neighbors(word='superb', mat=glv[0], rownames=glv[1], distfunc=cosine)[: 10]

## Matrix Reweighting

Taking a look at the similarity measurements listed above, we can see that they offer some insight into the embedded context contained within written language, but they're not quite as accurate as we would like them to be. This is where matrix reweighting comes into play. The goal of matrix reweighting is to amplify the important, trustworthy and unusual representations found within language, and de-emphasize the unimportant, and untrustworthy representations.

### Normalization

Here we see another form of normalization that can be applied to the data. With length_norm() seen above, we normalize using the vector_length(). There are other ways that we can normalize the datasets as well, such as each row by the sum of its values, which turns each row into a probability distribution over the columns :

In [ ]:
def prob_norm(u):
    return u / np.sum(u)

It should be noted, however, that these normalization measures are insensitive to the magnitude of the underlying counts, which can cause problems in larger datasets. An example is [1,10] is vastly different than [1000, 10000], and those differences will end up being obscured by these normalization procedures.

### Pointwise Mutual Information [1]

This brings us to something called Pointwise Mutual Information (PMI), which can help address the issue of the ignorance of the magnitude of the counts within the previous normalization functions. This is the measure between a pair of discreet outcomes $x$ and $y$, as defined as :

$$PMI(x,y) = log \frac{P(x,y)}{P(x) \cdot P(y)}$$

$PMI(w,c)$ measures the association between a word $w$ and a context $c$ by calculating the log of the ratio between their joint probability (the frequency in which they occur together) and their marginal probabilities (the frequency with which they occur independently).

This can be shown by considering the actual number of observations within a corpus :

$$PMI(w,c) = log \frac{\#(w,c) \cdot |D|}{\#(w) \cdot \#(c)}$$

The challenge with PMI arises when we compute $M^{PMI}$. This leaves us with rows containing many entries of word-context pairs $(w,c)$ that were never observed in the corpus, for which $PMI(w,c) = log 0 = -\infty$. This leaves the matrix 'ill-defined'. We could do potentially smooth over the probabilities using something like a Dirichlet prior by adding a small "fake" count to the underlying counts matrix. But this leaves us with, still, a very dense matrix to deal with.

An alternative approach would be to replace $M^{pmi}$ with $M_0^{PMI}$, in which $PMI(w,c) = 0$ in all cases $\#(w,c) = 0$, which results in a sparse matrix.

When approaching this problem, we understand that there are "bad" (uncorrelated) word-context pairs, which are represented with negative matrix entries, and replacing $M^{PMI}$ with $M_0^{PMI}$ will leave non-existent word context pairs with 0 entries in the matrix, which would rate them "better" than resulting negative entries from the real $M^{PMI}$.

### Positive Pointwise Mutual Information [1]

To get around the problem framed above, where uncorrelated word-context pairs will have negative matrix entries whereas zeroed out entries would exist for non-existent word-context pairs, we're able to a modification to the PMI algorithm called Positive Pointwise Mutual Information (PPMI). This will map all negative values within the matrix to 0.0, leveling the playing field between unseen word-context pairs and negative valued entries within the matrix. We can view this new positive PMI matrix as $M^{PPMI}$.

$$ PPMI(w,c) = max(PMI(w,c),0) $$

Below we capture both implementations, PMI and PPMI. The positive argument default to True, implementing PPMI, if this argument is set to False, it will perform regular PMI.

It's interesting to note that the PMI matrix $M^{PMI}$ is what emerges as the optimal solution for SGNS, as seen below.

In [ ]:
#PMI example
def pmi(mat=None, rownames=None, positive=True):
    # Create a joint probability table
    p = mat / np.sum(mat, axis=None)
    # Pre-compute column sums
    colprobs = np.sum(p, axis=0)
    # Vectorize this function so it can be applied rowwise
    np_pmi_log = np.vectorize((lambda x : _pmi_log(x, positive=positive)))
    p = np.array([np_pmi_log(row / (np.sum(row)*colprobs)) for row in p])
    return (p, rownames)

def _pmi_log(x, positive=True):
    """Where positive = False, return log(x) if possible, else 0.
    Where positive = True, log(x) is mapped to 0 where negative."""
    # set val var to 0.0
    val = 0.0
    # conditional 
    if x > 0.0:
        val = np.log(x)
    if positive:
        val = max([val, 0.0])
    return val
    

In [ ]:
# Process the ww data using PPMI
ww_ppmi = pmi(mat=ww[0], rownames=ww[1], positive=True)

In [ ]:
neighbors(word='superb', mat=ww_ppmi[0], rownames=ww_ppmi[1], distfunc=cosine)[: 10]

And again with Euclidean distance measure :

In [ ]:
neighbors(word='superb', mat=ww_ppmi[0], rownames=ww_ppmi[1], distfunc=euclidean)[: 10]

## Skipgram Negative Sampling (SGNS)

### Skipgram Model

The skipgram model assumes a corpus of words $w \in V_{w}$ and their contexts $c \in V_{c}$, where $V_w$ and $V_c$ are word and context vocabularies. The context vacabulary was assembled using an _L_-sized window $w_{i-L}...,w_{i-1}, w_{i+1}...,w_{i+L}$. 

The objective of the skipgram model is to maximize the average log probability, where $c$ is the size of the training context, seen as a function of the center word $w_{i}$.

As an example, say we set the window size to a count of 3, using the example sentence below, "The quick brown fox jumped over the lazy dog", if we choose 'jumped' as our word of interest, we can add it to $V_{w}$. After adding 'jumped' the the word vector, we can use the window function to capture the surrounding words by simply counting forward and backward through the sentence and append each word to $V_{c}$.

<img src="SGNSwindow.jpg">

We denote the collection of observed word context pairs, $(w,c)$, as $D$. We also use $\#(w,c)$ to denote the number of times observe $(w,c)$ in $D$.

### SGNS Objective

The objective of Skipgram with Negative Sampling is to maximize the the probability that $(w,c)$ came from the data $D$. This can be modeled as a distribution such that $P(D=1|w,c)$ be the probability that $(w,c)$ came from the data and $P(D=0|w,c) = 1 - P(D=1|w,c)$ the probability that $(w,c)$ did not. 

The distribution is modeled as :

$$P(D=1|w,c) = \sigma(\vec{w} \cdot \vec{c}) = \frac{1}{1+e^{-\vec{w} \cdot \vec{c}}}$$

where $\vec{w}$ and $\vec{c}$ (each a d-dimensional vector) are the model parameters to be learned.

The negative sampling tries to maximize $P(D=1|w,c)$ for observed $(w,c)$ pairs while maximizing $P(D=0|w,c)$ for stochastically sampled "negative" examples, under the assumpting that selecting a contect for a given word is likely to result in an unobserved $(w,c)$ pair.

SGNS's objective for a single $(w,c)$ observation is then:

$$ log \sigma(\vec{w} \cdot \vec{c}) + k \cdot \mathbb{E}_{c_{N} \sim P_{D}} [log \sigma(\vec{-w} \cdot \vec{c}_N)] $$

where $k$ is the number of "negative" samples and $c_{N}$ is the sampled context, drawn according to the empirical unigram distribution $P_{D}(c) = \frac{\#c}{|D|}$.

This object is then trained in an online fashion using stochastic gradient updated over the observed pairs in the corpus $D$. The goal objective then sums over the observed $(w,c)$ pairs in the corpus :

$$ \ell = \Sigma_{w \in V_{w}} \Sigma_{c \in V_{c}} \#(w,c)(log \sigma(\vec{w} \cdot \vec{c}) + k \cdot \mathbb{E}_{c_{N} \sim P_{D}} [log \sigma(\vec{-w} \cdot \vec{c}_N)]$$

Optimizing this objective groups words that have similar embeddings, while scattering unobserved pairs.

Tying the two of these together, we can see that SGNS with $k = 1$ is attempting to implicity factorize the familiar matrix $M^{PMI}$.

Past re-implementing word2vec and all of the hyperparameters that have been discussed here, we can defer to the word2vec implementation that exists within the library gensim.

Within gensim are two implementations of word2vec, both hierachical softmax and negative sampling. 

In order for us to be able to show how the skip-gram model, and ultimately negative sampling combined with skip-gram, works we will need an unprocessed dataset. We can see below how to import that and ready it for processing.

In [4]:
# Method used to import raw data
def import_raw_data(dir):
    corpora = []
    filenames = glob(dir)
    for line in fileinput.input(filenames):
        soup.findAll(line)
        corpora.append(line)
    return corpora

raw_comments = import_raw_data('aclImdb/train/unsup/*.txt')

AttributeError: 'str' object has no attribute 'descendants'

In [ ]:
example = BeautifulSoup(raw_comments[0], "html.parser")

print(raw_comments[0])

#print(example)

In [ ]:
sentences = MySentences('aclImdb/train/unsup/')

In [ ]:
model = gensim.models.Word2Vec(sentences)

## References

[1] Omar Levy, Yoav Goldberg. Neural Word embedding as Implicit Matrix Factorization. NIPS, 2015

[2] Omar Levy, Yoav Golderg. word2vec Explained: Deriving Mikolov et al.’s Negative-Sampling Word-Embedding Method

[3] Omar Levy, Yoav Goldber, Ido Dagan. Improving Distributional Similarity with Lessons Learned from Word Embeddings

[4] Tomas Mikolov, Ilya Sutskever, Kai Chen, Gregory S. Corrado, and Jeffrey Dean. Distributed representations of words and phrases and their compositionality.

In [ ]:
import sys; print(sys.version)